# 상장기업공시 정보 수집하기
[https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage](https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage)

In [1]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from myfunc import text_clean

In [3]:
# 기업 데이터가 없이 페이지 껍데기만 있는 주소
# url = "https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage"
# r = requests.get(url)
# print(r.status_code)
# soup = bs(r.content, "lxml")
# soup

In [4]:
url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
payload = dict(method="searchCorpList", pageIndex=1, currentPageSize=15, 
               orderMode=3, orderStat="D", searchType=13,
               fiscalYearEnd="all", location="all")
r = requests.get(url, params=payload)
print(r.url)
print(r.status_code)
soup = bs(r.content, "lxml")
soup

https://kind.krx.co.kr/corpgeneral/corpList.do?method=searchCorpList&pageIndex=1&currentPageSize=15&orderMode=3&orderStat=D&searchType=13&fiscalYearEnd=all&location=all
200


<html><body><section class="scrarea type-00">
<table class="list type-00 tmt30" summary="회사명, 업종, 주요제품, 상장일, 결산월, 대표자명, 홈페이지, 지역">
<caption>목록</caption>
<colgroup>
<col width="*"/>
<col width="15%"/>
<col width="20%"/>
<col width="11%"/>
<col width="8%"/>
<col width="10%"/>
<col width="8%"/>
<col width="12%"/>
</colgroup>
<thead>
<tr class="first" id="title-contents">
</tr>
</thead>
<tbody>
<tr class="first">
<td class="first" title="삼성기업인수목적13호(주)"><img alt="코스닥" class="vmiddle legend" src="/images/common/icn_t_ko.gif"/> <a href="#companysum" id="companysum" onclick="companysummary_open('0115H'); return false;" title="삼성스팩13호"> 삼성스팩13호</a> </td>
<td class="textOverflow" title="금융 지원 서비스업">금융 지원 서비스업</td>
<td class="textOverflow" title="기업인수합병">기업인수합병</td>
<td class="txc">2026-01-21</td>
<td class="txc">12월</td>
<td class="txc" title="성상환">성상환</td>
<td class="txc">
</td>
<td class="txc">서울특별시</td>
</tr>
<tr>
<td class="first" title="(주)엔비알모션"><img alt="코스닥" class="vmiddle legend" src="

In [18]:
# 컬럼명
col_names = soup.select_one("table.list.type-00.tmt30")["summary"].split(", ")
col_names

['회사명', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역']

In [63]:
all_result = []
for row in soup.select("table.list.type-00.tmt30 tbody tr"):
    result = {}
    for idx , (key, tds) in enumerate(zip(col_names, row.select("td"))):
#         print(tds.get_text(strip=True))
        
        if idx == 0:
            market = tds.select_one("img")['alt']
            code = tds.select_one("a")["onclick"].split("'")[1]+"0"
            company_name = tds.get_text(strip=True)
            result['종류'] = market
            result['종목코드'] = code
            result['회사명'] = company_name
#             print(market, code, company_name)
        elif key == '홈페이지':
            if tds.select_one("a") != None:
                result.setdefault(key, tds.select_one("a")['href'])
            else:
                result[key] = ""
        else:
            result.setdefault(key, tds.get_text(strip=True))
#     print(result)
    all_result.append(result)  

df = pd.DataFrame(all_result)
df

,종류,종목코드,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,코스닥,0115H0,삼성스팩13호,금융 지원 서비스업,기업인수합병,2026-01-21,12월,성상환,,서울특별시
1,코스닥,0004V0,엔비알모션,기타 금속 가공제품 제조업,강구 및 테이퍼롤러,2026-01-14,12월,문두성,,경상남도
2,코스닥,493330,지에프아이,일반 목적용 기계 제조업,"2차전지소화안전제품, 일반소화안전제품",2026-01-14,12월,이상섭,,경기도
3,코스닥,012210,삼미금속,기타 금속 가공제품 제조업,"자동차부품, 중장비부품, 선박부품, 방산부품 등",2025-12-29,12월,이남영,http://www.sammimetal.com,경상남도
4,코스닥,490470,세미파이브,"건축기술, 엔지니어링 및 관련 기술 서비스업",반도체 설계 서비스,2025-12-29,12월,조명현,http://semifive.com,경기도
5,코스닥,491000,리브스메드,의료용 기기 제조업,"다관절 다자유도 복강경 수술기구인 ArtiSential, ArtiSeal, Arti...",2025-12-24,12월,이정주,http://www.livsmed.com,경기도
6,코스닥,0097F0,미래에셋비전스팩10호,금융 지원 서비스업,기업인수합병,2025-12-23,12월,윤태원,,서울특별시
7,코스닥,0099W0,미래에셋비전스팩11호,금융 지원 서비스업,기업인수합병,2025-12-22,12월,김양태,,서울특별시
8,코스닥,0013V0,삼진식품,수산물 가공 및 저장 처리업,"반찬용 어묵, 국탕용 어묵 등 어묵관련 제품 판매",2025-12-22,12월,박용준,http://samjinfood.com,부산광역시
9,코넥스,059180,엔더블유시,통신 및 방송 장비 제조업,"FTTH Drop, FTTH 커넥터류, 5G CPRI, MDU, 광접속함체",2025-12-22,12월,김희중,,경기도


# 전체 데이터수집하기

In [1]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from myfunc import text_clean

In [83]:
all_result = []
page = 1
total_pages = 1

while True:
    
    if page > total_pages:
        break
    
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method="searchCorpList", pageIndex=page, currentPageSize=100, 
                   orderMode=3, orderStat="D", searchType=13,
                   fiscalYearEnd="all", location="all")
    r = requests.get(url, params=payload)
    print(r.url)
    print(r.status_code)
    soup = bs(r.content, "lxml")

    # 전체 페이지 구하기
    if total_pages == 1:
        n_items = int(soup.select_one(".info.type-00 em").get_text().replace(",", ""))
        if n_items % 100 == 0:
            total_pages = n_items // 100
        else:
            total_pages = n_items // 100 + 1

    print(f"{page}/{total_pages}   수집중", end="\r")
    
    # 1페이지 데이터 수집 코드    
    for row in soup.select("table.list.type-00.tmt30 tbody tr"):
        result = {}
        for idx , (key, tds) in enumerate(zip(col_names, row.select("td"))):
    #         print(tds.get_text(strip=True))

            if idx == 0:
                market = tds.select_one("img")['alt']
                code = tds.select_one("a")["onclick"].split("'")[1]+"0"
                company_name = tds.get_text(strip=True)
                result['종류'] = market
                result['종목코드'] = code
                result['회사명'] = company_name
    #             print(market, code, company_name)
            elif key == '홈페이지':
                if tds.select_one("a") != None:
                    result.setdefault(key, tds.select_one("a")['href'])
                else:
                    result[key] = ""
            else:
                result.setdefault(key, tds.get_text(strip=True))
    #     print(result)
        all_result.append(result)  

    page += 1
    
    time.sleep(3)
    
df = pd.DataFrame(all_result)
df    

https://kind.krx.co.kr/corpgeneral/corpList.do?method=searchCorpList&pageIndex=1&currentPageSize=100&orderMode=3&orderStat=D&searchType=13&fiscalYearEnd=all&location=all
200
https://kind.krx.co.kr/corpgeneral/corpList.do?method=searchCorpList&pageIndex=2&currentPageSize=100&orderMode=3&orderStat=D&searchType=13&fiscalYearEnd=all&location=all
200
https://kind.krx.co.kr/corpgeneral/corpList.do?method=searchCorpList&pageIndex=3&currentPageSize=100&orderMode=3&orderStat=D&searchType=13&fiscalYearEnd=all&location=all
200
https://kind.krx.co.kr/corpgeneral/corpList.do?method=searchCorpList&pageIndex=4&currentPageSize=100&orderMode=3&orderStat=D&searchType=13&fiscalYearEnd=all&location=all
200
https://kind.krx.co.kr/corpgeneral/corpList.do?method=searchCorpList&pageIndex=5&currentPageSize=100&orderMode=3&orderStat=D&searchType=13&fiscalYearEnd=all&location=all
200
https://kind.krx.co.kr/corpgeneral/corpList.do?method=searchCorpList&pageIndex=6&currentPageSize=100&orderMode=3&orderStat=D&searc

,종류,종목코드,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,코스닥,0115H0,삼성스팩13호,금융 지원 서비스업,기업인수합병,2026-01-21,12월,성상환,,서울특별시
1,코스닥,0004V0,엔비알모션,기타 금속 가공제품 제조업,강구 및 테이퍼롤러,2026-01-14,12월,문두성,,경상남도
2,코스닥,493330,지에프아이,일반 목적용 기계 제조업,"2차전지소화안전제품, 일반소화안전제품",2026-01-14,12월,이상섭,,경기도
3,코스닥,012210,삼미금속,기타 금속 가공제품 제조업,"자동차부품, 중장비부품, 선박부품, 방산부품 등",2025-12-29,12월,이남영,http://www.sammimetal.com,경상남도
4,코스닥,490470,세미파이브,"건축기술, 엔지니어링 및 관련 기술 서비스업",반도체 설계 서비스,2025-12-29,12월,조명현,http://semifive.com,경기도
...,...,...,...,...,...,...,...,...,...,...
2770,유가증권,000100,유한양행,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
2771,유가증권,000120,CJ대한통운,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
2772,유가증권,000050,경방,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
2773,유가증권,000700,유수홀딩스,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


28


# 데이터가 table 태그안에 있는 경우 쉽게 수집하기

In [122]:
df_list = []
for num in range(28):
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method="searchCorpList", pageIndex=1, currentPageSize=100, 
                   orderMode=3, orderStat="D", searchType=13,
                   fiscalYearEnd="all", location="all")
    r = requests.get(url, params=payload)
    df = pd.read_html(r.text)[0]
    df_list.append(df)
final_df = pd.concat(df_list)
final_df 

C:\Users\haram\AppData\Local\Temp\ipykernel_13856\3161106290.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(r.text)[0]
C:\Users\haram\AppData\Local\Temp\ipykernel_13856\3161106290.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(r.text)[0]
C:\Users\haram\AppData\Local\Temp\ipykernel_13856\3161106290.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(r.text)[0]
C:\Users\haram\AppData\Local\Temp\ipykernel_13856\3161106290.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,삼성스팩13호,금융 지원 서비스업,기업인수합병,2026-01-21,12월,성상환,NaN,서울특별시
1,엔비알모션,기타 금속 가공제품 제조업,강구 및 테이퍼롤러,2026-01-14,12월,문두성,NaN,경상남도
2,지에프아이,일반 목적용 기계 제조업,"2차전지소화안전제품, 일반소화안전제품",2026-01-14,12월,이상섭,NaN,경기도
3,삼미금속,기타 금속 가공제품 제조업,"자동차부품, 중장비부품, 선박부품, 방산부품 등",2025-12-29,12월,이남영,홈페이지 보기,경상남도
4,세미파이브,"건축기술, 엔지니어링 및 관련 기술 서비스업",반도체 설계 서비스,2025-12-29,12월,조명현,홈페이지 보기,경기도
...,...,...,...,...,...,...,...,...
95,우양에이치씨,"구조용 금속제품, 탱크 및 증기발생기 제조업","화공 플랜트, 에너지 플랜트",2025-03-28,12월,김진태,홈페이지 보기,경기도
96,더즌,전기 통신업,디지털뱅킹 솔루션(B2B 금융 서비스),2025-03-24,12월,조철한,홈페이지 보기,서울특별시
97,심플랫폼,소프트웨어 개발 및 공급업,NUBISON AIoT(산업용 데이터 수집과 산업용 AI서비스 운영),2025-03-21,12월,임대근..,홈페이지 보기,서울특별시
98,티엑스알로보틱스,일반 목적용 기계 제조업,"물류자동화, 로봇자동화, AMR/AGV",2025-03-20,12월,엄인섭,홈페이지 보기,경기도
